# Dimensionado de Apache

Un problema común en servidores Web es que un pico de usuarios lleve la carga del servidor a un punto donde se agote la memoria RAM del servidor y se comience a utilizar la memoria de intercambio o *SWAP*.

Al el SWAP estar basado en almacenamiento de disco duro, es varias órdenes de magnitud más lento que la memoria RAM por que la velocidad de respuesta de los procesos así como el acceso a dispositivos de almacenamiento bajan considerablemente.

Una vez en este estado incluso el conectarnos por SSH al servidor para detener el servicio o realizar reparación de emergencia podría verse seriamente afectado, por lo que la duración de la caída del sitio se maximiza y en ocasiones hay que optar por reiniciar el servidor ya sea físico o virtual.

Lo mejor es optimizar los servidores Web para que utilicen únicamente los recursos disponibles en el servidor y en una situación de sobre carga solamente una cantidad mínima de usuarios se verían afectados y nos permite inclusive crecer la granja de servidor en agregando más servidores Web evitando la interrupción a los usuarios.

En Apache hacemos uso de la configuración del **Multi Process Module** o **MPM**, siguiendo este proceso:

1. Quitamos módulos de Apache innecesarios, módulos de lenguaje dinámico (Php, Python en virtualenv, Npm en NodeJS, etc)

2. Medimos el peso (en memoria) del proceso de Apache

3. Calculamos cuantos procesos de Apache caben cómodamente en nuestro servidor Web

4. Realizamos pruebas de stress sobre el sitio, verificando que nuestros cálculos son correctos

## Instalación de Apache - mínimo

Vamos a realizar una instalación mínima de Apache, removiendo módulos innecesarios y utilizando el **MPM event**

In [ ]:
sudo apt-get -y install apache2

Vamos ahora a calcular el peso de el proceso de Apache.

**Importante**: Se debe calcular el peso de Apache cuando ha estado sirviendo contenido. El nuestro se encuentra "virgen" por lo que los resultados podrían ser menores.

Para ello vamos a utilizar la herramienta de ``ps(1)`` la cual lista procesos en sistemas Unix.

Para explicar la línea de comando completa, vamos a ir agregando porciones de código al uso básico de la herramienta ``ps(1)``.

Ejecute los siguientes comandos de forma individual, entendiendo la diferencia entre las ejecuciones y los cambios en la línea de comando de cada uno.

In [ ]:
# Forma tradicional de usar ps(1). Útil para listar todos los proceso del equipo en forma de árbol
ps faxw | head -20

In [ ]:
# ps(1) mostrando solo procesos de Apache
ps -C apache2

In [ ]:
# Ahora solicitamos mostrar la columna de RSS o Resident Set Size
ps -o rss -C apache2

# Por comparación, otras métricas disponibles en ps(1). RSS se considera la más precisa para nuestra necesidad.
ps -o size -C apache2
ps -o vsz -C apache2

In [ ]:
# Para manipular el número, queremos eliminar el encabezado de columnas
ps -o rss -C apache2 --no-headers

In [ ]:
# Como existen varios procesos de Apache, nótese que tienen diferentes tamaños de memoria.
# Por esta razón vamos a crear un promedio, aprovechando para traducir el valor a megabytes.
ps -o rss -C apache2 --no-headers | awk '{total+=$0} END {print total/NR/1024}'

In [ ]:
# Por último, vamos a guardar el valor dentro de una variable, para poder imprimir nuestros cálculos de Workers
httpd_mem=$(ps -o rss -C apache2 --no-headers | awk '{total+=$0} END {print total/NR/1024}')

echo $httpd_mem

In [ ]:
# Ahora podemos seguir utilizando el valor dentro de esta receta de Jupyter.

echo $httpd_mem

### Configuración de MPM

Dependiendo del tipo de worker que tengamos, podemos crear la configuración usando estas recetas.
s
Iniciamos por medir le memoria disponible, en megas, y guardando el valor en una variable.

**Importante**: En este tutorial vamos a asumir que si existe un componente de base de datos (PostgreSQL, MySQL, MongoDB, etc) se encuentra en un servidor aparte. Si ud corre la base de datos en el servidor, debe definir cuanta memoria va a dedicar a la base de datos, y elminar esta memoria de la memoria total, así como realizar mediciones para verificar que no se sobre pasa de la memoria asignada.

In [ ]:
mem_total=$(grep MemTotal /proc/meminfo | awk '{print $2/1024}')

echo $mem_total

In [ ]:
# Archivo /etc/apache2/mods-available/mpm_event.conf

# Event MPM
# StartServers: Cantidad de servidores que arrancan desde el inicio
# MinSpareThreads: Cantidad mímina de hilos a mentener en espera
# MaxSpareThreads: Cantidad máxima de hilos a mentener en espera
# ThreadsPerChild: Cantidad de hijos constant number of worker threads in each server process
# MaxRequestWorkers: Hilos máximos en total
# MaxConnectionsPerChild: Solicitudes máximas que puede resolver un proceso
echo "<IfModule mpm_event_module>
        StartServers                     2
        MinSpareThreads          25
        MaxSpareThreads          75
        ThreadLimit                      64
        ThreadsPerChild          25
        MaxRequestWorkers         $( echo $mem_total / $httpd_mem | bc )
        MaxConnectionsPerChild   0
</IfModule>"


In [ ]:
# Archivo /etc/apache2/mods-available/mpm_prefork.conf

# Prefork MPM
# StartServers: Cantidad de servidores que arrancan desde el inicio
# MinSpareThreads: Cantidad mímina de hilos a mentener en espera
# MaxSpareThreads: Cantidad máxima de hilos a mentener en espera
# ThreadsPerChild: Cantidad de hijos constant number of worker threads in each server process
# MaxRequestWorkers: Hilos máximos en total
# MaxConnectionsPerChild: Solicitudes máximas que puede resolver un proceso


echo "<IfModule mpm_prefork_module>
    StartServers             5
    MinSpareServers          5
    MaxSpareServers         10
    MaxRequestWorkers       $( echo $mem_total / $httpd_mem | bc )
    MaxConnectionsPerChild   0
</IfModule>"


Ahora que hizo un estimado de la cantidad de procesos máximos que puede correr con Apache, ahora realice cambios en Apache para medir cuanto sube o baja la memoria.

En el momento que cambie la configuración de Apache **reinicie** el servicio y ejecute de nuevo los cálculos de memoria en esta libreta.

Algunas sugerencias:

- Active Php integrado a Apache. Recuerdo que esto cambia del MPM "event" al MPM "prefork".
- Active Php-fpm o Php HHVM. En este caso, debe restar la memoria utilizar para Php, a la memoria disponible para Apache.
- Active y desactive módulos de Apache (autoindex, status, dir, ssl, etc)

Se recomienda también revisar la [guía de desempeño de Apache](https://httpd.apache.org/docs/2.4/misc/perf-tuning.html).